In [ ]:
import os
import json
import re
import path as path
output_file = path.wiki
root_dir = '../data/text'

# メモリ効率よくファイルを逐次的に処理
with open(output_file, 'w', encoding='utf-8') as json_file:
    #json_file.write('[')  # JSON配列の開始
    first_entry = True

    # root_dir内のすべてのサブディレクトリとファイルを探す
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            #json_file.write('[')
            #first_entry = True

            # ファイルを開いて、内容を処理
            with open(file_path, 'r', encoding='utf-8') as f:
                content = re.sub(r'\b(\w+)\b(?:\n{2,})\1\b', r'\1', f.read())

                content = re.sub(r'\n.*?\.', r'', content, re.MULTILINE)

                content = re.sub(r'\n', '', content)

                # docタグごとにJSONに変換
                docs = re.findall(r'<doc.*?>(.*?)</doc>', content, re.DOTALL)
                for doc in docs:
                    if not first_entry:
                        json_file.write('\n')  # 最初のエントリでない場合はカンマを付ける
                    json.dump({"text": doc.strip()}, json_file, ensure_ascii=False)
                    first_entry = False
            #json_file.write(']')
            #break
    #json_file.write(']')  # JSON配列の終了

トークナイザの語彙生成

In [ ]:
import path as path
from charTokenizer import CharTokenizer

tokenizer = CharTokenizer()
with open(path.wiki, 'r', encoding='utf-8') as f:
    for text in f:
        tokenizer.build_vocab(text)

print(tokenizer.vocab_size)

トークナイザの語彙をJSON形式で保存

In [ ]:
import json
with open(path.charTokenizer, 'w', encoding='utf-8') as f:
    json.dump(tokenizer.vocab, f, ensure_ascii=False)